# 1. Predicting Heart Disease Using a Classification Tree

## 1.1 Data Cleaning

In [12]:
library(tidyverse)   # Incluye dplyr, ggplot2, readr
library(readxl)      # Leer Excel 
library(fastDummies) # para crear dummies de categoricas
library(caret) # para dividir data en train y test
library(tree) # classification tre
library(grf) 

library(randomForest)  # alternativa y para importancia
library(rpart)
library(rpart.plot)
library(scales)

randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.


Adjuntando el paquete: 'randomForest'


The following object is masked from 'package:dplyr':

    combine


The following object is masked from 'package:ggplot2':

    margin



Adjuntando el paquete: 'scales'


The following object is masked from 'package:purrr':

    discard


The following object is masked from 'package:readr':

    col_factor




In [2]:
data <- read_csv("C:/Users/VICTOR/Documents/GitHub/Decision_Trees/R/input/processed.cleveland.data", col_names = FALSE)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [37]:
head(data)  

age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,hd
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0


In [24]:
names(data)

[1] "X1"  "X2"  "X3"  "X4"  "X5"  "X6"  "X7"  "X8"  "X9"  "X10" "X11" "X12"
[13] "X13" "X14"

In [3]:
nombres_nuevos <- c('age', 'sex', 'cp', 'restbp', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'hd')
colnames(data) <- nombres_nuevos
names(data)

[1] "age"     "sex"     "cp"      "restbp"  "chol"    "fbs"     "restecg"
 [8] "thalach" "exang"   "oldpeak" "slope"   "ca"      "thal"    "hd"

In [4]:
data <- data %>%
  mutate(across(where(is.character), ~ na_if(., "?")))
data <- data %>%
    drop_na()
data <- data %>%
  mutate(across(everything(), ~ as.numeric(.)))

In [5]:
str(data)

tibble [297 × 14] (S3: tbl_df/tbl/data.frame)
 $ age    : num [1:297] 63 67 67 37 41 56 62 57 63 53 ...
 $ sex    : num [1:297] 1 1 1 1 0 1 0 0 1 1 ...
 $ cp     : num [1:297] 1 4 4 3 2 2 4 4 4 4 ...
 $ restbp : num [1:297] 145 160 120 130 130 120 140 120 130 140 ...
 $ chol   : num [1:297] 233 286 229 250 204 236 268 354 254 203 ...
 $ fbs    : num [1:297] 1 0 0 0 0 0 0 0 0 1 ...
 $ restecg: num [1:297] 2 2 2 0 2 0 2 0 2 2 ...
 $ thalach: num [1:297] 150 108 129 187 172 178 160 163 147 155 ...
 $ exang  : num [1:297] 0 1 1 0 0 0 0 1 0 1 ...
 $ oldpeak: num [1:297] 2.3 1.5 2.6 3.5 1.4 0.8 3.6 0.6 1.4 3.1 ...
 $ slope  : num [1:297] 3 2 2 3 1 1 3 1 2 3 ...
 $ ca     : num [1:297] 0 3 2 0 0 0 2 0 1 0 ...
 $ thal   : num [1:297] 6 3 7 3 3 3 3 3 7 7 ...
 $ hd     : num [1:297] 0 2 1 0 0 0 3 0 2 1 ...


### Categoricas a transformar: 
    -cp
    -restecg
    -slope
    -ca
    -thal
    -hd

In [6]:
data <- data %>%
  mutate(y = ifelse(hd > 0, 1, 0))

In [7]:
variables_categoricas = c("cp", "restecg", "slope", "ca", "thal", "hd")
data <- data %>%
  mutate(across(all_of(variables_categoricas), ~ as.factor(.)))
data <- data %>%
    fastDummies::dummy_cols(select_columns = variables_categoricas, 
                              remove_selected_columns = TRUE,
                              remove_first_dummy = TRUE)

## 1.2 Data Analysis

In [8]:
set.seed(123)

train_index <- createDataPartition(data$y, p = 0.8, list = FALSE)
train <- data[train_index, ]
test  <- data[-train_index, ]

train$y <- factor(train$y)
test$y  <- factor(test$y)

tree_model <- tree(y ~ ., data = train)

predictions <- predict(tree_model, test, type = "class")

mean(predictions == test$y)

labels <- c("Does not have HD", "Has HD")
table(Real = factor(test$y, levels = c(0,1), labels = labels),
      Pred = factor(predictions, levels = c(0,1), labels = labels))

confusionMatrix(predictions, test$y,
                dnn = c("Predicho", "Real"),
                mode = "everything",
                positive = "1")

[1] 1

                  Pred
Real               Does not have HD Has HD
  Does not have HD               32      0
  Has HD                          0     27

Confusion Matrix and Statistics

        Real
Predicho  0  1
       0 32  0
       1  0 27
                                     
               Accuracy : 1          
                 95% CI : (0.9394, 1)
    No Information Rate : 0.5424     
    P-Value [Acc > NIR] : < 2.2e-16  
                                     
                  Kappa : 1          
                                     
 Mcnemar's Test P-Value : NA         
                                     
            Sensitivity : 1.0000     
            Specificity : 1.0000     
         Pos Pred Value : 1.0000     
         Neg Pred Value : 1.0000     
              Precision : 1.0000     
                 Recall : 1.0000     
                     F1 : 1.0000     
             Prevalence : 0.4576     
         Detection Rate : 0.4576     
   Detection Prevalence : 0.4576     
      Balanced Accuracy : 1.0000     
                                     
       'Positive' Class : 1          
                                   

> El modelo clasificó perfectamente a todos los individuos: 100% de precisión, sensibilidad y especificidad.

# 2. Causal Forest 

In [14]:
set.seed(123)
data <- read_csv("C:/Users/VICTOR/Documents/GitHub/Decision_Trees/R/input/processed.cleveland.data", col_names = FALSE)
nombres_nuevos <- c('age', 'sex', 'cp', 'restbp', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'hd')
colnames(data) <- nombres_nuevos
names(data)
data <- data %>%
  mutate(across(where(is.character), ~ na_if(., "?")))
data <- data %>%
    drop_na()
data <- data %>%
  mutate(across(everything(), ~ as.numeric(.)))
data <- data %>%
  mutate(y = ifelse(hd > 0, 1, 0))
variables_categoricas = c("cp", "restecg", "slope", "ca", "thal", "hd")
data <- data %>%
  mutate(across(all_of(variables_categoricas), ~ as.factor(.)))


covs <- data %>% select(-y, -hd, everything()) %>% select(-starts_with("hd")) 

data <- data %>%
  mutate(age = as.numeric(age),
         sex = as.numeric(as.character(sex)),    # si era factor
         restbp = as.numeric(restbp),
         oldpeak = as.numeric(oldpeak))

# Tratamiento
n <- nrow(data)
data <- data %>%
  mutate(T = rbinom(n, 1, 0.5))

# Creamos Y según la regla:
data <- data %>%
  mutate(eps = rnorm(n),
         Y = (1 + 0.05 * age + 0.3 * sex + 0.2 * restbp) * T + 0.5 * oldpeak * eps) %>%
  select(-eps)  

# Estimamos efecto por OLS simple (regresión Y ~ T) en todo el dataset
ols_fit <- lm(Y ~ T, data = data)
summary(ols_fit)



Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "age"     "sex"     "cp"      "restbp"  "chol"    "fbs"     "restecg"
 [8] "thalach" "exang"   "oldpeak" "slope"   "ca"      "thal"    "hd"


Call:
lm(formula = Y ~ T, data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-10.1552  -0.7442   0.1079   0.6764  15.7747 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -0.1079     0.2274  -0.475    0.635    
T            30.2439     0.3266  92.604   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 2.813 on 295 degrees of freedom
Multiple R-squared:  0.9667,	Adjusted R-squared:  0.9666 
F-statistic:  8575 on 1 and 295 DF,  p-value: < 2.2e-16


In [ ]:
# Estimamos efectos heterogéneos con causal_forest (grf)
X <- data %>% select(age, sex, restbp, chol, thalach, oldpeak) %>% as.matrix()
Yvec <- data$Y
W <- data$T

set.seed(123)
cf <- causal_forest(X, Yvec, W, num.trees = 2000)  #

# Predecir CATE (heterogeneous treatment effects) para todos los individuos
cf_pred <- predict(cf, estimate.variance = TRUE)
data <- data %>% mutate(cate_cf = cf_pred$predictions,
                        cate_cf_se = sqrt(pmax(0, cf_pred$variance)))  # var could be tiny negative due to num errors

# --- 4. Alternativa: Estimar CATE por diferencia de Random Forests (Treated - Control)
# Ajustar 2 bosques: uno para tratados, otro para controles
set.seed(123)
rf_t <- randomForest(x = X[W == 1, , drop = FALSE], y = Yvec[W == 1], ntree = 500)
rf_c <- randomForest(x = X[W == 0, , drop = FALSE], y = Yvec[W == 0], ntree = 500)

pred_t_all <- predict(rf_t, X)
pred_c_all <- predict(rf_c, X)
data <- data %>% mutate(cate_rf_diff = pred_t_all - pred_c_all)

# --- 5. Plot representative tree of predicted CATEs (usamos cate_cf o cate_rf_diff)
# Usaremos 'cate_cf' si disponible; ajustamos un rpart con maxdepth = 2 para interpretabilidad
library(rpart)
set.seed(123)
tree_cate <- rpart(cate_cf ~ age + sex + restbp + chol + thalach + oldpeak,
                   data = data,
                   control = rpart.control(maxdepth = 2, cp = 0.00001, minsplit = 20))
rpart.plot(tree_cate, main = "Árbol representativo de CATE (maxdepth=2)")

# Interpretación: ver nodos y medias de cate_cf por terminal.

# --- 6. Importancias de features
# Opción: usar randomForest para predecir cate_cf desde X y extraer importance
set.seed(123)
rf_cate <- randomForest(x = X, y = data$cate_cf, importance = TRUE, ntree = 500)
imp <- importance(rf_cate, type = 1) # tipo 1: mean decrease accuracy
imp_df <- data.frame(variable = rownames(imp), importance = imp[,1]) %>%
  arrange(desc(importance))

# Plot importancias
ggplot(imp_df, aes(x = reorder(variable, importance), y = importance)) +
  geom_col() +
  coord_flip() +
  labs(title = "Importancia de features para predecir CATE (randomForest)",
       x = "", y = "Importancia (mean decrease accuracy)")

# --- 7. Distribución de covariables por terciles de efecto
# Standardizar covariables
covs_standard <- data %>%
  select(age, sex, restbp, chol, thalach, oldpeak) %>%
  mutate(across(everything(), ~ as.numeric(scale(.)))) # escala z

# Añadir cate y tercil
data2 <- bind_cols(data, covs_standard) %>%
  mutate(cate_tercile = ntile(cate_cf, 3)) %>%
  mutate(cate_tercile_label = factor(cate_tercile, labels = c("low", "medium", "high")))

# Calcular medias de cada covariables por tercil
means_by_tercile <- data2 %>%
  group_by(cate_tercile_label) %>%
  summarise(across(c(age, sex, restbp, chol, thalach, oldpeak), ~ mean(.x, na.rm = TRUE))) %>%
  pivot_longer(-cate_tercile_label, names_to = "variable", values_to = "mean_z")

# Heatmap (terciles en y, variables en x)
ggplot(means_by_tercile, aes(x = variable, y = cate_tercile_label, fill = mean_z)) +
  geom_tile() +
  scale_fill_gradient2(low = "blue", mid = "white", high = "red", midpoint = 0) +
  labs(title = "Heatmap: medias estandarizadas de covariables por terciles de CATE",
       x = "Covariable", y = "Tercil de efecto (CATE)") +
  theme_minimal()

# --- 8. Resúmenes rápidos: medias y tamaños de terciles
data2 %>% group_by(cate_tercile_label) %>% summarise(n = n(), mean_cate = mean(cate_cf), sd_cate = sd(cate_cf))